# Analysis of Amazon Product Data

After construction of the graph (product-id and link to cross-buys), the following topics will be investigated:
* correlation between in-degrees and salesrank
* and some further smart questions... influence of number of reviews?, are there different sub-graphs like produkt categories?
* 
* 
* 
Data source: https://snap.stanford.edu/data/amazon-meta.html

Data import

In [3]:
# Import graphframes (from Spark-Packages)

from graphframes import *
from pyspark.sql.functions import monotonically_increasing_id, desc


# Get the product data (already pre-processed data in table format)
products = spark.read.csv("/FileStore/tables/ProductData_small.csv", sep=";", header=True)

# The the cross-buy data (similar products)
links = spark.read.csv("/FileStore/tables/LinkData_small.csv", sep=";", header=True)


quick check of the data formats

In [5]:
display(products.take(3))

Id,ASIN,Group,Salesrank,Reviews,Title
21,0790747324,DVD,795,140,The Time Machine
84,B000063W82,Video,780,14,The Best of Schoolhouse Rock! - 30th Anniversary Edition
252,B0000262WI,Music,581,82,The Köln Concert


In [6]:
display(links.take(3))

src,dest
0790747324,B00007JMD8
0790747324,B00004RF9B
0790747324,B00005JKFR


The graph will be constructed based on the product data. Each product is identified through its unique *ASIN* (Amazon Standard Identification Number) code

GraphFrame library required *ASIN* to be renamed to *id*, but be careful, there is already an "Id" in the DataFrame

In [8]:
# rename "Id" to "Id-nu" to avoid any confusion
# rename "ASIN" to "id"
ProductVertices = products.withColumnRenamed("Id", "Id-nu").distinct()
ProductVertices = ProductVertices.withColumnRenamed("ASIN", "id").distinct()

# check if done correctly
display(ProductVertices.take(3))

Id-nu,id,Group,Salesrank,Reviews,Title
13260,B000002TIR,Music,2004,16,French Kiss
13683,0140281916,Book,2877,16,Bargaining for Advantage : Negotiation Strategies for Reasonable People
30773,B000001EYO,Music,3579,31,The Big Lebowski: Original Motion Picture Soundtrack


How does the Edge table look like?

In [10]:
display(links.take(1))

src,dest
0790747324,B00007JMD8


Although not always strictly required, it's good practice to have an ID column for the tables. 

Since the product links does not have one yet, create and additional ID:

In [12]:
# 2. Create Edges
links = links.withColumn("link-id", monotonically_increasing_id())

display(links.take(3))

src,dest,link-id
0790747324,B00007JMD8,0
0790747324,B00004RF9B,1
0790747324,B00005JKFR,2


Salesrank and number of reviews are read as string. Change type to int for calculations.

In [14]:
# cast the arrival delay column 'ARR_DELAY' from String to int and rename the column to 'delay'
# TODO: put in the new column name below
ProductVertices = ProductVertices.withColumn('Salesrank', ProductVertices.Salesrank.cast('int'))
ProductVertices = ProductVertices.withColumn('Reviews', ProductVertices.Reviews.cast('int'))
display(ProductVertices.take(1))

Id-nu,id,Group,Salesrank,Reviews,Title
13260,B000002TIR,Music,2004,16,French Kiss


Rename links to similar products ar *src* and *dst*

In [16]:
# rename destination "dest" to "dst"
links = links.withColumnRenamed('dest', 'dst')

# let's keep only the most important columns from the whole dataframe
#tripEdges = departureDelays.select("tripid", "src", "dst", "origin_city_name", "dest_city_name", "delay")


Now comes the moment of truth! Let's see if the graph can be constructed!

In [18]:
# 2. Build the GraphFrame based on products and links

ProductGraph = GraphFrame(ProductVertices, links)

# print a sample of the vertices and edges
print("Products: %d" % ProductGraph.vertices.count())
print
print("SAMPLE PRODUCT DATA:")
print(ProductGraph.vertices.take(1))
print

print("Links: %d" % ProductGraph.edges.count())
print
print("SAMPLE LINK DATA:")
print(ProductGraph.edges.take(1))
print

Products: 10611
SAMPLE PRODUCT DATA:
[Row(Id-nu='13260', id='B000002TIR', Group='Music', Salesrank=2004, Reviews=16, Title='French Kiss')]
Links: 21560
SAMPLE LINK DATA:
[Row(src='0790747324', dst='B00007JMD8', link-id=0)]
 Out[ 14 ]: <function print>

The Graph is constructed. Do the fancy analyses!

1) assess the number of in-degress per product and compare it to the salesrank

In [21]:
display(ProductGraph.inDegrees.sort('inDegree', ascending=False))

#inDegRank = ProductGraph.inDegrees.sort('inDegree', ascending=False)

id,inDegree
B00008LDNZ,78
B000068DBC,37
6304765266,36
B00005ATZT,34
B00001QEE7,33
6305368171,33
B00003CXA2,30
B000059HB7,29
630522577X,29
B000022TSH,27


In [22]:
maxNumInDeg = ProductGraph.inDegrees.groupBy().max('inDegree')
display(maxNumInDeg)

maxNumInDegValue = str(maxNumInDeg.collect()[0]['max(inDegree)'])
#print(maxNumInDegValue)

maxInDegreeProduct = ProductGraph.inDegrees.filter('inDegree = {}'.format(maxNumInDegValue))
display(maxInDegreeProduct)


id,inDegree
B00008LDNZ,78


In [23]:
# join back to get product info based on max inDegree
ProductMaxInDegree = ProductGraph.vertices.join(maxInDegreeProduct, 'id')
display(ProductMaxInDegree)




id,Id-nu,Group,Salesrank,Reviews,Title,inDegree
B00008LDNZ,548091,DVD,110,83,Laura,78


In [24]:
# repeat exercise from above but with entire list
selection = ProductGraph.inDegrees.sort('inDegree', ascending=False)
#display(selection)

ProductMaxInDegFull = ProductGraph.vertices.join(selection, 'id').sort('inDegree', ascending=False)
display(ProductMaxInDegFull)

id,Id-nu,Group,Salesrank,Reviews,Title,inDegree
B00008LDNZ,548091,DVD,110,83,Laura,78
B000068DBC,546689,DVD,109,633,Pulp Fiction (Collector's Edition),37
6304765266,97579,DVD,351,152,While You Were Sleeping,36
B00005ATZT,547040,DVD,1020,140,The Fugitive (Special Edition),34
6305368171,276371,DVD,236,500,You've Got Mail,33
B00001QEE7,104775,DVD,49,160,The Little Mermaid (Limited Issue),33
B00003CXA2,547272,DVD,185,530,Forrest Gump,30
B000059HB7,4388,DVD,1877,79,Rio Bravo,29
630522577X,548182,DVD,253,180,My Fair Lady,29
0684801523,199628,Book,956,934,The Great Gatsby,27
